In [1]:
import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib.dates as mdates
import numpy as np
#from datetime import datetime
#from datetime import date

# See what products didn't sell in 2021

inv = pd.read_csv('InventoryList_2022-02-04.csv',header=0,dtype=str)

inv = inv[['ProductCode','Name','Category','Barcode','PriceTier1','Status']].copy()
inv = inv[inv.Status == 'ACTIVE']
inv.head()

sold = pd.read_excel('2021details.xlsx',engine='openpyxl',header=0,skiprows=5, parse_dates=False)
custs = ['Customer Square ' + str(x) for x in range(1,4)] #we have 3 stores active, we won't count the Warehouse
retail_names = ['Airport','Pleasant Valley','Oak Hill'] # make sure this matches the above list length - no known automation possible

sold = sold[sold.Customer.isin(custs) & sold.Product.notnull()].reset_index(drop=True) #remove wholesale and weird null anomoly entries
sold = sold.drop(['COGS','Profit'], axis=1, inplace=False)#not trusted data columns or useful, we'll ignore them for now

quick = sold.groupby(['Product','SKU','Customer']).Quantity.sum().reset_index()
#quick.info()

dead_map = []

for i, row in inv[inv.Status == 'ACTIVE'].iterrows():
    prod = row.Name
    for cust in custs:
        if(quick[(quick.Product == prod) & (quick.Customer == cust)].empty):
            dead_map.append([prod,cust,row.Category,row.PriceTier1])
            
#print(dead_map)

output = pd.DataFrame(dead_map,columns=['Product','Customer','Category','Retail'])
output

output.to_csv('NeverSoldFULL.csv',index=False)

# Do it again but more smarter, eventually run a longer report

In [2]:
df = pd.read_csv('AvailabilityReport_2022-02-04.csv',header=0,dtype=str)

df = df[['Category','SKU','ProductName','Location','Status','OnHand']].copy()
df.OnHand = df.OnHand.astype(float)
df.OnHand = df.OnHand.astype(int)
df.rename(columns={'ProductName':'Product'},inplace=True)
df = df[df.Location != 'Warehouse']

retail_names = ['Airport','Pleasant Valley','Oak Hill'] # make sure this matches the above list length - no known automation possible
custs = ['Customer Square ' + str(x) for x in range(1,len(retail_names)+1)] #we have 3 stores active, we won't count the Warehouse
store_map = dict(zip(retail_names,custs))
df['Customer'] = df.apply(lambda row: store_map[row.Location],axis=1)

#df[(df.Status == 'DEPRECATED') & (df.Location == 'Customer Square 3')].head()#good, its empty
df.head()

,Category,SKU,Product,Location,Status,OnHand,Customer
0,Non Alcoholic Beverage,777823111113,1 Water 355ml,Airport,ACTIVE,123,Customer Square 1
1,Non Alcoholic Beverage,777823111113,1 Water 355ml,Oak Hill,ACTIVE,1,Customer Square 3
2,Non Alcoholic Beverage,777823111113,1 Water 355ml,Pleasant Valley,ACTIVE,5,Customer Square 2
3,Bar Tools,1234500000000,12/24 Mixing Tin Sets,Airport,ACTIVE,0,Customer Square 1
4,Bar Tools,1234500000000,12/24 Mixing Tin Sets,Pleasant Valley,ACTIVE,0,Customer Square 2


In [4]:
#df[(df.Product.str.contains('Del Maguey')) & (df.Location == 'Oak Hill')]

,Category,SKU,Product,Location,Status,OnHand,Customer
4992,Mezcal,618397110020,Del Maguey Arroqueno 750ml,Oak Hill,ACTIVE,2,Customer Square 3
4996,Mezcal,618397201391,Del Maguey Barril 750ml,Oak Hill,ACTIVE,4,Customer Square 3
4999,Mezcal,618398080384,Del Maguey Chichicapa 750ml,Oak Hill,ACTIVE,2,Customer Square 3
5003,Mezcal,618397020015,Del Maguey Crema de Mezcal 750ml,Oak Hill,ACTIVE,3,Customer Square 3
5007,Mezcal,618397201018,Del Maguey Espadin Especial 750ml,Oak Hill,ACTIVE,3,Customer Square 3
5010,Mezcal,618397201384,Del Maguey Iberico Pechuga 750ml,Oak Hill,ACTIVE,2,Customer Square 3
5013,Mezcal,618397201674,Del Maguey Jabali 750ml,Oak Hill,ACTIVE,3,Customer Square 3
5017,Mezcal,618397201902,Del Maguey Las Milpas 750ml,Oak Hill,ACTIVE,1,Customer Square 3
5021,Mezcal,618397201308,Del Maguey Madrecuixe 750ml,Oak Hill,ACTIVE,2,Customer Square 3
5025,Mezcal,618397960021,Del Maguey Minero 750ml,Oak Hill,ACTIVE,2,Customer Square 3


In [14]:
sale = pd.read_excel('last365.xlsx',engine='openpyxl',header=0,skiprows=5, parse_dates=False)#sales by product summary last 365 no totals export
sale = sale.drop(['Unit','Invoice','Profit'],axis=1).reset_index()#COGS too
sale = sale[(sale.Customer.isin(custs)) & (sale.Product.notnull())]
sale['Unit'] = sale.Sale / sale.Quantity
sale.head()

,index,Product,Category,Customer,Quantity,Sale,COGS,Unit
2,2,1 Water 355ml,Non Alcoholic Beverage,Customer Square 1,6,9.00,1.373,1.50
3,3,1 Water 355ml,Non Alcoholic Beverage,Customer Square 2,1,1.50,0.500,1.50
5,5,123 Organic Anejo Tequila 750ml,Tequila,Customer Square 1,3,179.97,129.370,59.99
6,6,123 Organic Anejo Tequila 750ml,Tequila,Customer Square 2,5,299.95,226.500,59.99
7,7,123 Organic Anejo Tequila 750ml,Tequila,Customer Square 3,1,59.99,41.240,59.99


In [17]:
#sale[(sale.Product.str.contains('Del Maguey')) & (sale.Customer == 'Customer Square 3')]

In [18]:
master = df.merge(right=sale,on=['Product','Customer'],how='outer')
master.drop(['index'],axis=1,inplace=True)
master.Unit = master.apply(lambda row: master[master.Product == row.Product].Unit.max(),axis=1)
master["StockValue"] = master.OnHand * master.Unit
master = master.reindex(columns=['SKU','Product','Customer','Location','Status','Category_x','Category_y','OnHand','Quantity','Sale','Unit','StockValue'])
master.rename(columns={'Category_x':'CategoryDear','Category_y':'CategorySale'},inplace=True)
master.iloc[3:10]

,SKU,Product,Customer,Location,Status,CategoryDear,CategorySale,OnHand,Quantity,Sale,Unit,StockValue
3,1234500000000,12/24 Mixing Tin Sets,Customer Square 1,Airport,ACTIVE,Bar Tools,NaN,0.0,NaN,NaN,NaN,NaN
4,1234500000000,12/24 Mixing Tin Sets,Customer Square 2,Pleasant Valley,ACTIVE,Bar Tools,NaN,0.0,NaN,NaN,NaN,NaN
5,7503013136221,123 Organic Anejo Tequila 750ml,Customer Square 1,Airport,ACTIVE,Tequila,Tequila,0.0,3.0,179.97,59.99,0.00
6,7503013136221,123 Organic Anejo Tequila 750ml,Customer Square 3,Oak Hill,ACTIVE,Tequila,Tequila,0.0,1.0,59.99,59.99,0.00
7,7503013136221,123 Organic Anejo Tequila 750ml,Customer Square 2,Pleasant Valley,ACTIVE,Tequila,Tequila,1.0,5.0,299.95,59.99,59.99
8,7503013136207,123 Organic Blanco Tequila 750ml,Customer Square 1,Airport,ACTIVE,Tequila,Tequila,2.0,3.0,143.97,47.99,95.98
9,7503013136207,123 Organic Blanco Tequila 750ml,Customer Square 3,Oak Hill,ACTIVE,Tequila,NaN,1.0,NaN,NaN,47.99,47.99


In [19]:
master = master[(master.OnHand > 0) | (master.Quantity.notnull()) & (master.SKU.notnull())] #get rid of things we probably just don't support anymore and stupid anomolies
master.iloc[3:10]

,SKU,Product,Customer,Location,Status,CategoryDear,CategorySale,OnHand,Quantity,Sale,Unit,StockValue
5,7503013136221,123 Organic Anejo Tequila 750ml,Customer Square 1,Airport,ACTIVE,Tequila,Tequila,0.0,3.0,179.97,59.99,0.00
6,7503013136221,123 Organic Anejo Tequila 750ml,Customer Square 3,Oak Hill,ACTIVE,Tequila,Tequila,0.0,1.0,59.99,59.99,0.00
7,7503013136221,123 Organic Anejo Tequila 750ml,Customer Square 2,Pleasant Valley,ACTIVE,Tequila,Tequila,1.0,5.0,299.95,59.99,59.99
8,7503013136207,123 Organic Blanco Tequila 750ml,Customer Square 1,Airport,ACTIVE,Tequila,Tequila,2.0,3.0,143.97,47.99,95.98
9,7503013136207,123 Organic Blanco Tequila 750ml,Customer Square 3,Oak Hill,ACTIVE,Tequila,NaN,1.0,NaN,NaN,47.99,47.99
10,7503013136207,123 Organic Blanco Tequila 750ml,Customer Square 2,Pleasant Valley,ACTIVE,Tequila,Tequila,3.0,10.0,479.90,47.99,143.97
11,7503013136214,123 Organic Reposado Tequila 750ml,Customer Square 1,Airport,ACTIVE,Tequila,Tequila,0.0,3.0,170.97,56.99,0.00


In [20]:
#jared = smart[smart.Customer == 'Customer Square 3'].groupby('Category').Product.count()
dead_cats = master.groupby(['CategoryDear','Customer']).CategorySale.count()
total_cats = master.groupby(['CategoryDear','Customer']).CategoryDear.count()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(jared)
#print(dead_cats,"\n",total_cats)

In [26]:
master[(master.OnHand > 2) & (master.Quantity.isnull()) & (master.Location == 'Oak Hill')].to_csv('tempOHout.csv')

In [21]:
t1 = dead_cats.to_frame()
t1 = t1.rename(columns={'CategorySale':'Category'})

t2 = total_cats.to_frame()
t2 = t2.rename(columns={'CategoryDear':'Category'})

cats = t1.merge(right=t2,on=['CategoryDear','Customer'],suffixes=('_Sold', '_Total'))
cats = cats.reset_index()
cats.head()

,CategoryDear,Customer,Category_Sold,Category_Total
0,Absinthe,Customer Square 1,13,15
1,Absinthe,Customer Square 2,19,22
2,Absinthe,Customer Square 3,10,21
3,Amaro,Customer Square 1,91,103
4,Amaro,Customer Square 2,99,111


In [22]:
#now we create a column of savings for if we dropped all the dead weight per row to show savings
cats["Savings"] = cats.apply(lambda row: master[(master.CategoryDear == row.CategoryDear) & (master.Customer == row.Customer) & (master.Quantity.isnull())].StockValue.sum(),axis=1)
cats.to_csv('DeadCats.csv',index=False)

In [23]:
cats.groupby('Customer').Savings.sum()
#cats.Savings.sum()/1.35

Customer
Customer Square 1     26882.219282
Customer Square 2     47029.700936
Customer Square 3    103812.004897
Name: Savings, dtype: float64